# Unsteady 2D cylinder

This tutorial shows how to run a 2D unsteady simulation of a cylinder.

## 1. Create Project from volume mesh
- Load Python libraries and Flow360 client. If you use environment variables or tokens, initialize them here so later API calls can authenticate.
- Project is created from a volume mesh

In [1]:
import flow360 as fl
from flow360.examples import Cylinder2D

Cylinder2D.get_files()


project = fl.Project.from_volume_mesh(
    Cylinder2D.mesh_filename, name="Unsteady 2D Cylinder from Python"
)

vm = project.volume_mesh

Output()

Output()

[17:47:54] INFO: VolumeMesh successfully submitted:                                                                
                   type   = Volume Mesh                                                                            
                   name   = Unsteady 2D Cylinder from Python                                                       
                   id     = vm-cf43d640-0fb0-44b8-8c39-3d026f7529ed                                                
                   status = uploaded                                                                               
           

Output()

## 2. Define time settings
To run an unsteady simulation, we need to define a step size and the total number of steps. Additionally, it is also possible to define the maximum number of pseudo steps done between two physical steps.

In [2]:
with fl.SI_unit_system:
    time_settings = fl.Unsteady(
            step_size=2,
            steps=20,
            max_pseudo_steps=40,)

[17:48:13] INFO: using: SI unit system for unit inference.

## 3. Define boundary conditions and solver settings
The models section includes the boundary conditions definition and the solver settings.

In [3]:
with fl.SI_unit_system:
    models=[
        fl.Fluid(
            navier_stokes_solver=fl.NavierStokesSolver(
                absolute_tolerance=1e-9, linear_solver=fl.LinearSolver(max_iterations=25)
            ),
            turbulence_model_solver=fl.NoneSolver(),
        ),
        fl.Wall(surfaces=[vm["fluid/wall"]]),
        fl.Freestream(surfaces=[vm["fluid/farfield"]]),
        fl.SlipWall(surfaces=[vm["fluid/periodic_0_l"], vm["fluid/periodic_0_r"]]),
    ]

           INFO: using: SI unit system for unit inference.

## 4. Define SimulationParams
- The simulation parameters are defined in the python class fl.SimulationParams()
- THe previous settings are included as well as the operating conditions, outputs and reference geometries.

In [4]:
with fl.SI_unit_system:
    params = fl.SimulationParams(
        operating_condition=fl.AerospaceCondition.from_mach_reynolds(
            reynolds_mesh_unit=50, mach=0.1, project_length_unit=fl.u.m
        ),
        models=models,
        time_stepping=time_settings,
        outputs=[
            fl.SurfaceOutput(output_fields=["Cp"], surfaces=[vm["*"]]),
            fl.VolumeOutput(
                output_fields=[
                    "primitiveVars",
                    "vorticity",
                    "residualNavierStokes",
                    "T",
                    "Cp",
                    "mut",
                ],
            ),
        ],
        reference_geometry=fl.ReferenceGeometry(
            area=20, moment_center=[0, 0, 0], moment_length=[1, 1, 1]
        ),
    )

           INFO: using: SI unit system for unit inference.

           INFO: Density and viscosity were calculated based on input data, ThermalState will be automatically     
           created.

## 5. Run Case

In [5]:
case = project.run_case(params, "Unsteady 2D Cylinder case from Python")

           INFO: Density and viscosity were calculated based on input data, ThermalState will be automatically     
           created.

           INFO: using: SI unit system for unit inference.

[17:48:15] INFO: Successfully submitted:                                                                           
                   type   = Case                                                                                   
                   name   = Unsteady 2D Cylinder case from Python                                                  
                   id     = case-e96c2490-48ea-4f96-b99b-1a191d63c342                                              
                   status = pending                                                                                
           